## Exploring the Dataset

The speech samples are already divied into training, development, and test spilts. The splits are made in such way that evaluation speakers are not present in training split. You should use the splits as they are.

In [1]:
import pandas as pd

In [2]:
# read tsv file into a dataframe
sdr_df = pd.read_csv('SDR_metadata.tsv', sep='\t', header=0, index_col='Unnamed: 0')

In [3]:
sdr_df.head()

,identifier,speaker,split,label,file
0,5_theo_23,theo,TRAIN,5,speech_data/5_theo_23.wav
1,2_yweweler_39,yweweler,TEST,2,speech_data/2_yweweler_39.wav
2,6_yweweler_34,yweweler,DEV,6,speech_data/6_yweweler_34.wav
3,6_yweweler_16,yweweler,DEV,6,speech_data/6_yweweler_16.wav
4,9_yweweler_2,yweweler,TEST,9,speech_data/9_yweweler_2.wav


In [13]:
# Count labels in each set.
dev_label_counts = sdr_df.where(sdr_df.split == "DEV").label.value_counts()
train_label_counts = sdr_df.where(sdr_df.split == "TRAIN").label.value_counts()
test_label_counts = sdr_df.where(sdr_df.split == "TEST").label.value_counts()
# Make a dataframe with the counts that sorts columns by their numerical value
label_counts = (pd.DataFrame([dev_label_counts, train_label_counts, test_label_counts], index=["DEV", "TRAIN", "TEST"])
                .sort_index(axis=1)
                .astype(int))
# Add a column with the total and a row with the total
label_counts["TOTAL"] = label_counts.sum(axis=1)
label_counts.loc["TOTAL"] = label_counts.sum(axis=0)

label_counts

,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,TOTAL
DEV,47,45,46,54,54,50,54,53,44,50,497
TRAIN,200,200,200,200,200,200,200,200,200,200,2000
TEST,53,55,54,46,46,50,46,47,56,50,503
TOTAL,300,300,300,300,300,300,300,300,300,300,3000
